In [56]:
import numpy as np
import pandas as pd

In [57]:
def dist_meters(x1,y1,x2,y2,wind): #distance between two points in the direction of the wind
    a = (x2-x1)*np.cos(np.deg2rad(wind+90)) #add 90 due to orientation 
    b = (y2-y1)*np.sin(np.deg2rad(wind+90))
    return (a+b)*140 #140 is the distance lengths in matrix
    

In [58]:
def second_val(x1,y1,x2,y2,wind):
    #x1,y1 WT1 location
    #x2,y2 WT2 location
    #wind -wind dir in degrees
    r = 75 #150m is the projected diameters for upcoming iterations of off-shore turbines
    z0 = 0.04 #wake spreading constant (0.04 recommended for offshore windfarms)
    return (r/(r+(z0*abs(dist_meters(x1,y1,x2,y2,wind)))))**2

In [59]:
def first_val(sp):
    #sp = windspeed in m/s
    V = sp * (1.583314688)
    C = ((5.47581845 * (10**(-6)) * (V**5.00641402))+1.132584887)**(-1) #Ad modeled by Gualtieri

    return (1-np.sqrt(1-C))

In [60]:
def third_val(x1,y1,x2,y2,wind):
    Ar = 17700 # Rotor swept area of turbine (circumference given the radius)
    Dij = dist_meters(x1,y1,x2,y2,wind)
    if Dij < 0:
        return 0 #given turbine is in the opposite direction of wake from main turbine
    RWij = 75 + (0.04*Dij)
    ri = 75
    DWRij = np.sqrt( (  ((x1)-(x2))**2) + ( ((y1)-(y2))**2) - (Dij/140)**2 ) * 140

    if RWij - ri >= DWRij:
        return 1
    elif RWij + ri <= DWRij:
        return 0
    
    THETAWij = 2*np.arccos(( DWRij**2 + (RWij**2 - ri**2))/(2*DWRij*RWij))
    THETArij = 2*np.arccos(( DWRij**2 - (RWij**2 - ri**2))/(2*DWRij*ri))

    left_term = RWij**2 * (THETAWij - np.sin(THETAWij))
    right_term = ri**2 * (THETArij - np.sin(THETArij))
    Ao = 0.5*(left_term + right_term)

    return Ao/Ar

In [61]:
def interaction(x1,y1,x2,y2,wind,sp):
    return first_val(sp)*second_val(x1,y1,x2,y2,wind)*third_val(x1,y1,x2,y2,wind)


In [65]:
wind = pd.read_csv('wind.csv')

In [75]:

for x in range(50):
    for y in range(50):
        mat = np.zeros((50,50))
        for ind,val in wind.iterrows():
            dir = val['dir']
            sp = val['speed']
            prob = val['prob']

            for i in range(50):
                for j in range(50):
                    
                    if not (i == x and j == y):
                        
                        mat[j,i] += prob * interaction(x,y,i,j,dir,sp)
        np.savetxt("interactionMats/"+str(x)+"_"+str(y)+".csv", mat, delimiter=",")

0.003350066055345979

350

1